<html>
    
    <center><h1>Coursera</h1></center>
    <center><h2>Applied Data Science Capstone Course</h2></center>
    <center><h2>Toronto Neighborhood Segmentation Project</h2></center>

    <h5>Author: Royden Lynch<br>
        Since:  04/16/2019</h5>
        
    <h4>Description:</h4>
    <p>
        This notebook will be the main notebook used for the week three project concerning segmentation of the                 neighborhoods in the city of toronto.
    </p>

</html>

### 1.0 - Project Setup

#### 1.1 - Install Packages

In [1]:
# BeautifulSoup
#   This will be used for handling the web-scraping portion
#   of the assignment. It provides many useful methods for
#   scrabing data from html files or from the html of pages
#   on the internet.
from bs4 import BeautifulSoup as BS
## Geocoder
#   This will be used to find the latitude and longitude
#   coordinates of a given postal code.
import geocoder
# Pandas
#   This will be used for handling data in the format of
#   dataframes. It provides many useful methods for dealing
#   with tabular-like data.
import pandas as pd
# Requests
#   This will be used for handling requests. This includes
#   both requests made to get html source of websites to
#   scrape data from, as well as, to get api calls from
#   our location data provider, Foursquare.
import requests

### 2.0 - Data Preprocessing

#### 2.1 - Retrieving Data from Source

In [2]:
# Get HTML Source File of Wikipedia Page
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [3]:
# Create BeautifulSoup Object
soup = BS(source, "lxml")
# Print the HTML
#print(soup.prettify())

In [4]:
# Find the Main Table
table = soup.find("table")
# Print the Table Found
#print(table.prettify())

In [5]:
# Create Empty DataFrame, with Specified Columns
column_names = ["PostCode", "Borough", "Neighborhood"]
to_hoods = pd.DataFrame(columns=column_names)

In [6]:
# For Each Row in the Table Body
for row in table.tbody.find_all("tr"):
    
    # Variables to Store Row Data
    row_data = []
    
    # For Each Column in the Row
    for column in row.find_all("td"):
        
        # Store the Column's Data (Text)
        col_data = column.text
            
        # Otherwise, Add the Data to the Row Data
        row_data.append(col_data)
        
    # Sanity Check for Empty Rows
    if len(row_data) == 0:
        continue

    # Create Row Data Dictionary
    row_data = {
        "PostCode"     : row_data[0],
        "Borough"      : row_data[1],
        "Neighborhood" : row_data[2][0:-1]
    }
    
    # If Borough is Undefined, Forget About Data    
    if row_data["Borough"] == "Not assigned":
        continue
        
    # If Neighborhood is Undefined, Let It Equal
    if row_data["Neighborhood"] == "Not assigned":
        row_data["Neighborhood"] = row_data["Borough"]
            
    # Append the Row of Data to the DataFrame
    to_hoods = to_hoods.append(row_data, ignore_index=True)

# Show the Resulting DataFrame
to_hoods.head(10)

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### 2.2 - Formatting Data

In [7]:
# Create Copy the Original DataFrame
to_hoods_by_PostCode = to_hoods.copy()

# Groupy th Neighborhoods (by PostCode/Borough) and Combine the Neighborhood as a List
to_hoods_by_PostCode = to_hoods_by_PostCode.groupby(["PostCode", "Borough"])["Neighborhood"].apply(list).reset_index()

# Convert List Obj to String
to_hoods_by_PostCode["Neighborhood"] = to_hoods_by_PostCode["Neighborhood"].apply(lambda x: str(x).strip("[]").replace("'",""))

# Show Head of Data
to_hoods_by_PostCode.head()


,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
to_hoods_by_PostCode.shape

(103, 3)

#### 2.3 - Retrieving Additional Data

In [9]:
# The function defined below represents how I attempted to retrieve the
# latitude and longitude data using the python geocoder package. However,
# as mentioned by the assignment's description, the package is apparantly
# very unstable and thus unreliable. I think my code is correct, but for
# some reason the package is not finding any coordinates.
# Therefore, I put the code in an unused function, rather than deleting it.
def add_latlng_coords_from_geocoder():
    # Create DataFrame for Latitude and Longitude Coordinates
    latlng_df = pd.DataFrame(columns=["Latitude", "Longitude"])

    # For Each Row in the to_hoods_by_PostCode DataFrame
    for index,row in to_hoods_by_PostCode.iterrows():
        print("next row")
        # Find Coords at the Post Code
        coords_ll = None
        while (coords_ll is None):
            g = geocoder.google("{}, Toronto, Ontario".format(row["PostCode"]))
            coords_ll = g.latlng
        # Append the Coords to the latlng_df DataFrame
        toAppend = {
            "Latitude"  : coords_ll[0],
            "Longitude" : coords_ll[1]
        }
        latlng_df = latlng_df.append(toAppend, ignore_index=True)

    # Show Resulting DataFrame
    latlng_df

    # Concatonate the latlng_df DataFrame to the Side of the to_hoods_by_PostCode DataFrame
    pd.concat([to_hoods_by_PostCode, latlng_df], axis=1)

In [10]:
# Read Latitude and Longitude Data from CSV Given by Assignment
to_PostCode_with_LatLng = pd.read_csv("Geospatial_Coordinates.csv")
print(to_PostCode_with_LatLng.shape)
to_PostCode_with_LatLng.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# It seems that the data was given to us by the prompt
# is in the same order as our DataFrame from step one.
# Therefore, I will make this as an assumption and just
# concatonate the two dataframes. If this was false, then
# I would need to go back and sort the latitude/longitude
# DataFrame and remove uneccessary PostalCodes.
to_hoods_by_PostCode_with_LatLng = pd.concat([to_hoods_by_PostCode, to_PostCode_with_LatLng["Latitude"], to_PostCode_with_LatLng["Longitude"]], axis=1)
to_hoods_by_PostCode_with_LatLng.head()


,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
